### This project is to load the SandP CSV data from data.world website and transform the data to required data types and to load into the DB table

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
csv_file = "../Resources/sp.csv"
sp_data_df = pd.read_csv(csv_file)
sp_data_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,1-Oct-20,"3,380.80","3,385.87","3,397.18","3,361.39",-,0.53%
1,30-Sep-20,"3,363.00","3,341.21","3,393.56","3,340.47",-,0.83%
2,29-Sep-20,"3,335.47","3,350.92","3,357.92","3,327.54",-,-0.48%
3,28-Sep-20,"3,351.60","3,333.90","3,360.74","3,332.91",-,1.61%
4,25-Sep-20,"3,298.46","3,236.66","3,306.88","3,228.44",-,1.60%


In [3]:
new_sp_data_df = sp_data_df[['Date', 'Price', 'Open', 'High', 'Low', 'Change %']].copy()
new_sp_data_df.head()

,Date,Price,Open,High,Low,Change %
0,1-Oct-20,"3,380.80","3,385.87","3,397.18","3,361.39",0.53%
1,30-Sep-20,"3,363.00","3,341.21","3,393.56","3,340.47",0.83%
2,29-Sep-20,"3,335.47","3,350.92","3,357.92","3,327.54",-0.48%
3,28-Sep-20,"3,351.60","3,333.90","3,360.74","3,332.91",1.61%
4,25-Sep-20,"3,298.46","3,236.66","3,306.88","3,228.44",1.60%


In [4]:
new_sp_data_df = new_sp_data_df.rename(columns={"Date": "trans_date",
                                        "Price": "price",
                                        "Open": "openvalue",
                                        "High": "dayhighvalue",
                                        "Low": "daylowvalue",
                                        "Change %": "percent_change"})
new_sp_data_df.drop_duplicates("trans_date", inplace=True)

In [77]:
new_sp_data_df['price']=new_sp_data_df['price'].astype('string')
new_sp_data_df['price']=new_sp_data_df['price'].str.replace(',','')
new_sp_data_df['price']=new_sp_data_df['price'].astype('float')

new_sp_data_df['openvalue']=new_sp_data_df['openvalue'].astype('string')
new_sp_data_df['openvalue']=new_sp_data_df['openvalue'].str.replace(',','')
new_sp_data_df['openvalue']=new_sp_data_df['openvalue'].astype('float')

new_sp_data_df['dayhighvalue']=new_sp_data_df['dayhighvalue'].astype('string')
new_sp_data_df['dayhighvalue']=new_sp_data_df['dayhighvalue'].str.replace(',','')
new_sp_data_df['dayhighvalue']=new_sp_data_df['dayhighvalue'].astype('float')

new_sp_data_df['daylowvalue']=new_sp_data_df['daylowvalue'].astype('string')
new_sp_data_df['daylowvalue']=new_sp_data_df['daylowvalue'].str.replace(',','')
new_sp_data_df['daylowvalue']=new_sp_data_df['daylowvalue'].astype('float')

#new_sp_data_df['percent_change']=new_sp_data_df['percent_change'].astype('string')
#new_sp_data_df['percent_change']=new_sp_data_df['percent_change'].str.replace(',','')
#new_sp_data_df['percent_change']=new_sp_data_df['percent_change'].astype('float')


#new_sp_data_df['price'] = pd.to_text(new_sp_data_df['price'])
#new_sp_data_df['price']=new_sp_data_df['price'].replace({'price': '^.,$'}, {'price': ''}, regex=True)

In [78]:
new_sp_data_df.head()

,trans_date,price,openvalue,dayhighvalue,daylowvalue,percent_change
0,1-Oct-20,3380.80,3385.87,3397.18,3361.39,0.53%
1,30-Sep-20,3363.00,3341.21,3393.56,3340.47,0.83%
2,29-Sep-20,3335.47,3350.92,3357.92,3327.54,-0.48%
3,28-Sep-20,3351.60,3333.90,3360.74,3332.91,1.61%
4,25-Sep-20,3298.46,3236.66,3306.88,3228.44,1.60%


In [79]:
rds_connection_string = "postgres:admin@localhost:5432/finance_db"
#<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [80]:
engine.table_names()

['sandp']

In [81]:
new_sp_data_df.to_sql(name='sandp', con=engine, if_exists='append', index=False)

In [82]:
pd.read_sql_query('select * from sandp', con=engine).head()

,trans_date,price,openvalue,dayhighvalue,daylowvalue,percent_change
0,2020-10-01,3380.80,3385.87,3397.18,3361.39,0.53%
1,2020-09-30,3363.00,3341.21,3393.56,3340.47,0.83%
2,2020-09-29,3335.47,3350.92,3357.92,3327.54,-0.48%
3,2020-09-28,3351.60,3333.90,3360.74,3332.91,1.61%
4,2020-09-25,3298.46,3236.66,3306.88,3228.44,1.60%
